In [1]:
import sympy as sym
import numpy as np
#import pylab as pl
from IPython.display import display, Math, Latex
sym.init_printing(use_unicode=True)
import sympy.utilities.autowrap as symauto

epss,Dels,nus,ta,tb,pa,pb,kappa,kappb,kas,kbs =\
sym.symbols('epsilon Delta nus ta tb pa pb kappaa kappab kas kbs',real=True)
kapp1a,kapp1b,Las,Lbs = \
sym.symbols('kappa1a kappa1b Las Lbs',real=True)

#Functions for evaluation
Ema = sym.exp(sym.I*(-nus*ta+pa))
Emb = sym.exp(sym.I*(-nus*tb+pb))
Epa = sym.exp(sym.I*(nus*ta+pa))
Epb = sym.exp(sym.I*(nus*tb+pb))
dma = kappa-sym.I*(nus-Dels) + kas*Epa         #d_{-,a}(\nu)
dmb = kappb-sym.I*(nus+Dels) + kbs*Epb         #d_{-,b}(\nu)
dpa = kappa-sym.I*(nus+Dels) + kas*sym.conjugate(Ema)   #d_{+,a}(\nu)
dpb = kappb-sym.I*(nus-Dels) + kbs*sym.conjugate(Emb)   #d_{+,b}(\nu)
Lab = sym.Abs(epss)**2 - dpa*dmb          #\Lambda_{ab}(\nu)
Lba = sym.Abs(epss)**2 - dpb*dma          #\Lambda_{ba}(\nu)

#Expectation value of the outgoing photon number
nom_ba = (kappa+kas*sym.cos(pa-nus*ta)-kapp1a*Las)*(kappb+kbs*sym.cos(pb+nus*tb))
nom_ab = (kappb+kbs*sym.cos(pb-nus*tb)-kapp1b*Lbs)*(kappa+kas*sym.cos(pa+nus*ta))
I_pexp  = 4*epss**2*(nom_ba/(Lab*sym.conjugate(Lab))+nom_ab/(Lba*sym.conjugate(Lba)))
I_mexp  = 4*epss**2*(nom_ba/(Lab*sym.conjugate(Lab))-nom_ab/(Lba*sym.conjugate(Lba)))

diffp   = sym.Rational(1,2)*(sym.diff(I_pexp,pa)-sym.diff(I_pexp,pb))
diffm   = sym.Rational(1,2)*(sym.diff(I_mexp,pa)-sym.diff(I_mexp,pb))
#display(sym.re(diffm))
#display(diffp)
#derivp = sym.lambdify((nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,ta,tb,phia,phib,La,Lb), diffp, modules="numpy")
derivm = sym.lambdify((nus,epss,Dels,kappa,kappb,kapp1a,kapp1b,kas,kbs,ta,tb,pa,pb,Las,Lbs), diffm, modules="numpy")
#num    = sym.lambdify((nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,ta,tb,phia,phib,La,Lb), I_pexp, modules="numpy")

In [2]:
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
import scipy.integrate as integ
from IPython.core.debugger import Tracer
#Tracer()() #this one triggers the debugger

def sens(x,delta,kapa,kapb,ra,rb,kta,ktb,phia,phib,La,Lb):
# Function to determine the phase sensitivity of the setup
    import time
    start  = time.clock()

    ## Parameters from inputs ##
    kapa  = kapa*2*np.pi
    kapb  = kapb*2*np.pi
    La    = La/100.
    Lb    = Lb/100.
    phia  = phia*np.pi
    phib  = phib*np.pi
    taua  = kta/kapa
    taub  = ktb/kapa
    Del   = delta*kapa
    eps   = x*kapa
    kap1a = kapa * ra                         #\kappa_{1,a} (coupling on the right hand side in mode a)
    kap1b = kapb * rb                         #\kappa_{1,b} (coupling on the right hand side in mode b)
    kap2a = kapa * (1-ra)                     #\kappa_{2,a} (coupling on the left hand side in mode a)
    kap2b = kapb * (1-rb)                     #\kappa_{2,b} (coupling on the left hand side in mode b)

    ka = 2*np.sqrt(ra*(1-ra)*(1-La))*kapa    #k_a (feedback strength in mode a)
    kb = 2*np.sqrt(rb*(1-rb)*(1-Lb))*kapb    #k_b (feedback strength in mode b)
    
    ### EXPRESSIONS ###
    def Ei(nu,t,phi):
        return np.exp(1j*(nu*t+phi))
    def d(nu,t,phi,kap,Del,k): #dpa if Del&phi, dma if -Del&-phi, dmb if Del&-phi, dpb if -Del&phi
        return kap - 1j*(nu+Del) + k*Ei(nu,t,-phi)
    def Lf(nu,Del,tp,phip,kapp,kp,tm,phim,kapm,km):#Lab if Del&p is a
        return abs(eps)**2 - d(nu,tp,phip,kapp,Del,kp)*d(nu,tm,-phim,kapm,Del,km)
    def albe(nu,t,phi,kap1,kap2,L): #alpha if quantities for a 1 is order is kept
        return np.sqrt(2*kap1) + np.sqrt(2*kap2*(1-L))*Ei(nu,t,phi)

    def D(nu,Del,tp,phip,kapp,kp,tm,phim,kapm,km,kap1,kap2,L): #Da if p is for b
        return np.sqrt(1-L)*Lf(nu,-Del,tp,phip,kapp,kp,tm,phim,kapm,km)*Ei(nu,tm,phim) +\
        d(nu,tp,phip,kapp,-Del,kp)*albe(nu,tm,phim,kap1,kap2,L)*albe(nu,tm,phim,kap2,kap1,L)

    def E(nu,Del,tp,phip,kapp,kp,tm,phim,kapm,km,kap1,kap2,L): # Da if p is for b
        return np.sqrt(L)*(np.sqrt(2*kap2)*d(nu,tp,phip,kapp,-Del,kp)*albe(nu,tm,phim,kap2,kap1,L) +\
        Lf(nu,-Del,tp,phip,kapp,kp,tm,phim,kapm,km))

    def K(nu,t,phi,kap,k,kap1,L):
        return kap + k*np.cos(phi+nu*t) - kap1*L
    
    def U1(nu,tL,phiL,kapL,kL,kap1L,L,t,phi,kap,k): #in case of ab L is for a
        return 4*K(-nu,tL,phiL,kapL,kL,kap1L,L)*K(nu,t,phi,kap,k,0,0) 
    def U2(nu,Del,tp,phip,kapp,kp,tm,phim,kapm,km,kap1,kap2,L): # in case of ab p is for b
        return np.abs(D(nu,Del,tp,phip,kapp,kp,tm,phim,kapm,km,kap1,kap2,L))**2 + \
    np.abs(E(nu,Del,tp,phip,kapp,kp,tm,phim,kapm,km,kap1,kap2,L))**2
    def U3(nu,Del,tp,phip,kapp,kp,kap1p,kap2p,Lp,tm,phim,kapm,km,kap1m,kap2m,Lm): # in case of ab p is for b
        return albe(-nu,tm,phim,kap2m,kap1m,Lm)*\
    (albe(nu,tp,phip,kap1p,kap2p,Lp)*np.conj(D(nu,Del,tm,phim,kapm,km,tp,phip,kapp,kp,kap1p,kap2p,Lp)) +\
    np.sqrt(2*kap2p*Lp)*np.conj(E(nu,Del,tm,phim,kapm,km,tp,phip,kapp,kp,kap1p,kap2p,Lp)))
    def U4(nu,Del,tp,phip,kapp,kp,kap1p,kap2p,Lp,tm,phim,kapm,km,kap1m,kap2m,Lm): # in case of ab p is for b
        return albe(-nu,tp,phip,kap2p,kap1p,Lp)*\
    (D(nu,Del,tp,phip,kapp,kp,tm,phim,kapm,km,kap1m,kap2m,Lm)*np.conj(albe(nu,tm,phim,kap1m,kap2m,Lm)) +\
    np.sqrt(2*kap2m*Lm)*np.conj(E(nu,Del,tp,phip,kapp,kp,tm,phim,kapm,km,kap1m,kap2m,Lm)))

    #Expectation value of the outgoing photon number
    def I_exp(nu):
        return np.real(eps**2*\
    (U1(nu,taua,phia,kapa,ka,kap1a,La,taub,phib,kapb,kb)/(np.abs(Lf(nu,Del,taua,phia,kapa,ka,taub,phib,kapb,kb))**2) + 
     U1(nu,taub,phib,kapb,kb,kap1b,Lb,taua,phia,kapa,ka)/(np.abs(Lf(nu,-Del,taub,phib,kapb,kb,taua,phia,kapa,ka))**2)))

#    plt.plot(np.linspace(-25,25,10000),np.real(derivm(np.linspace(-25*2*np.pi,25*2*np.pi,10000),\
#                                                       eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb)))
    # There is 10e-13 worth of imaginary part at the singularities
    plt.show()
    
    def T11(nu):
        return np.real(eps**2*U1(nu,taua,phia,kapa,ka,kap1a,La,taub,phib,kapb,kb)/\
    (np.abs(Lf(nu,Del,taua,phia,kapa,ka,taub,phib,kapb,kb))**2)/\
    derivm(nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    def T12(nu):
        return np.real(eps**2*U2(nu,Del,taub,phib,kapb,kb,taua,phia,kapa,ka,kap1a,kap2a,La)/\
    (np.abs(Lf(nu,-Del,taub,phib,kapb,kb,taua,phia,kapa,ka))**2)/\
    derivm(nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    def T21(nu):
        return np.real(eps**2*U3(nu,Del,taub,phib,kapb,kb,kap1b,kap2b,Lb,taua,phia,kapa,ka,kap1a,kap2a,La)/\
    (np.abs(Lf(nu,Del,taua,phia,kapa,ka,taub,phib,kapb,kb))**2)/\
    derivm(nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    def T22(nu):
        return np.real(eps**2*U4(nu,Del,taub,phib,kapb,kb,kap1b,kap2b,Lb,taua,phia,kapa,ka,kap1a,kap2a,La)/\
    (np.abs(Lf(nu,-Del,taub,phib,kapb,kb,taua,phia,kapa,ka))**2)/\
    derivm(nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    def T31(nu):
        return np.real(eps**2*U1(nu,taub,phib,kapb,kb,kap1b,Lb,taua,phia,kapa,ka)/\
    (np.abs(Lf(nu,-Del,taub,phib,kapb,kb,taua,phia,kapa,ka))**2)/\
    derivm(nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    def T32(nu):
        return np.real(eps**2*U2(nu,-Del,taua,phia,kapa,ka,taub,phib,kapb,kb,kap1b,kap2b,Lb)/\
    (np.abs(Lf(nu,Del,taua,phia,kapa,ka,taub,phib,kapb,kb))**2)/\
    derivm(nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    def T41(nu):
        return np.real(eps**2*U3(nu,-Del,taua,phia,kapa,ka,kap1a,kap2a,La,taub,phib,kapb,kb,kap1b,kap2b,Lb)/\
    (np.abs(Lf(nu,-Del,taub,phib,kapb,kb,taua,phia,kapa,ka))**2)/\
    derivm(nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    def T42(nu):
        return np.real(eps**2*U4(nu,-Del,taua,phia,kapa,ka,kap1a,kap2a,La,taub,phib,kapb,kb,kap1b,kap2b,Lb)/\
    (np.abs(Lf(nu,Del,taua,phia,kapa,ka,taub,phib,kapb,kb))**2)/\
    derivm(nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))

    T11_int,T11_err = integ.quad(T11,-np.inf,np.inf)
    T12_int,T12_err = integ.quad(T12,-np.inf,np.inf)
    T21_int,T21_err = integ.quad(T21,-np.inf,np.inf)
    T22_int,T22_err = integ.quad(T22,-np.inf,np.inf)
    T31_int,T31_err = integ.quad(T31,-np.inf,np.inf)
    T32_int,T32_err = integ.quad(T32,-np.inf,np.inf)
    T41_int,T41_err = integ.quad(T41,-np.inf,np.inf)
    T42_int,T42_err = integ.quad(T42,-np.inf,np.inf)

    T1 = T11_int*T12_int
    T2 = T21_int*T22_int
    T3 = T31_int*T32_int
    T4 = T41_int*T42_int

    sens = T1+T2+T3+T4
    sens_err = T11_err+T12_err+T21_err+T22_err+T31_err+T32_err+T41_err+T42_err
    
    N_res,N_err     = integ.quad(I_exp,-np.inf,np.inf)
#    print('The phase sensitivity is %.7f +/-' % (sens),sens_err)
#    print('The uncertainty of output is %.4f +/-' % (uncert_res),uncert_err)
#    print('The output particle number is %.7f +/-' % (N_res),N_err)
        
    end = time.clock()
    time = end-start
    mins = int(time/60.)
    secs = time-mins*60
#    print('The elapsed time: %d mins and %d secs' % (mins,secs))
    return sens
    #sens(x,delta,kapa,kapb,ra,rb,kta,ktb,phia,phib,La,Lb):    
#sens(0.6,0.,2.5,10,0.25,0.5,1.8833,1.8833,0.,-0.,10,10)
#sens0 = sens(0.1,0,10,10,0.5,0.5,1.8833,1.8833,0,0,0,0)
#senseps = np.zeros(100)
#sensDel = np.zeros(100)
sensPhi = np.zeros(10)
for i in range(0,10):
    sensPhi[i] = sens(0.6,0,10,10,0.5,0.5,1.8833,1.8833,i/100.,-i/100.,0,0)
#    senseps[i] = sens((i+1)/100.,0,10,10,0.5,0.5,1.8833,1.8833,0,0,0,0)
#    sensDel[i] = sens(0,2*(i+1)/100.,10,10,0.5,0.5,1.8833,1.8833,0,0,0,0)
plt.plot(np.linspace(0,1,10),sensPhi)
plt.show()
#sens0

/home/nnem614/Documents/anaconda3/lib/python3.5/site-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)
/home/nnem614/Documents/anaconda3/lib/python3.5/site-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


In [35]:
np.zeros(4)

array([ 0.,  0.,  0.,  0.])